In [ ]:
! pip install --upgrade \
  diffusers~=0.16 \
  transformers~=4.28 \
  safetensors~=0.3 \
  sentencepiece~=0.1 \
  accelerate~=0.18 \
  bitsandbytes~=0.38 \
  torch~=2.0 -q

In [ ]:
#upload files for DeepFloyd
from google.colab import files
uploaded = files.upload()

In [ ]:
import json
with open("/content/test_set_prompts.json", "r") as file:
  prompts = json.load(file)

In [ ]:
!pip install huggingface_hub --upgrade

In [ ]:
# workaround for memory issues/constraints with DeepFloyd
!pip install git+https://github.com/AndrewZhang126/accelerate

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from transformers import T5EncoderModel

text_encoder = T5EncoderModel.from_pretrained(
    "DeepFloyd/IF-I-XL-v1.0",
    subfolder="text_encoder", 
    device_map="auto", 
    load_in_8bit=True, 
    variant="8bit"
)

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained(
    "DeepFloyd/IF-I-XL-v1.0", 
    text_encoder=text_encoder, # pass the previously instantiated 8bit text encoder
    unet=None, 
    device_map="auto"
)

In [ ]:
import os

In [ ]:
images_path = '/content/'
dir_orig = "/content/orig/"
if not os.path.exists(dir_orig):
  os.makedirs(dir_orig)

file_names = os.listdir(images_path)
prompts_list = []
files_list = []

for i in range(len(file_names)):
  if len(file_names[i]) == 26 and file_names[i][0] == "0":
    files_list.append(file_names[i])

In [ ]:
for i in range(len(files)):
  image = Image.open(images_path + files_list[i])
  image.save(dir_orig + files_list[i])
  image_id = files_list[i][0:22]
  prompt = prompts[image_id]
  prompt_embeds, negative_embeds = pipe.encode_prompt(prompt)
  prompts_list.append((prompt_embeds, negative_embeds))

In [ ]:
import shutil

In [ ]:
shutil.make_archive('/content/orig', 'zip', '/content/orig')
from google.colab import files
files.download("/content/orig.zip")

In [ ]:
# free up memory
del text_encoder
del pipe

In [ ]:
import gc
import torch

def flush():
  gc.collect()
  torch.cuda.empty_cache()

In [ ]:
flush()

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    "DeepFloyd/IF-I-XL-v1.0", 
    text_encoder=None, 
    variant="fp16", 
    torch_dtype=torch.float16, 
    device_map="auto"
)

In [ ]:
from diffusers.utils import pt_to_pil

In [ ]:
dir_gen1 = "/content/genvr1/"
if not os.path.exists(dir_gen1):
  os.makedirs(dir_gen1)

v1_images = []
for i in range(len(prompts_list)):
  prompt_embeds, negative_embeds = prompts_list[i]
  generator = torch.Generator().manual_seed(1)
  image = pipe(
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=negative_embeds, 
    output_type="pt",
    generator=generator,
    ).images
    
  pil_image = pt_to_pil(image)
  #pipe.watermarker.apply_watermark(pil_image, pipe.unet.config.sample_size)
  gen_image = pil_image[0]
  gen_image.save(dir_gen1 + files_list[i][:-4] + "genvr1.jpg")
  v1_images.append(image)

In [ ]:
shutil.make_archive('/content/genvr1', 'zip', '/content/genvr1')
files.download("/content/genvr1.zip")

In [ ]:
del pipe
flush()

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    "DeepFloyd/IF-II-L-v1.0", 
    text_encoder=None, # no use of text encoder => memory savings!
    variant="fp16", 
    torch_dtype=torch.float16, 
    device_map="auto"
)

In [ ]:
dir_gen2 = "/content/genvr2/"
if not os.path.exists(dir_gen2):
  os.makedirs(dir_gen2)

v2_images = []

for i in range(len(prompts_list)):
  v1_image = v1_images[i]
  prompt_embeds, negative_embeds = prompts_list[i]
  image = pipe(
    image=v1_image, 
    prompt_embeds=prompt_embeds, 
    negative_prompt_embeds=negative_embeds, 
    output_type="pt",
    generator=generator,
  ).images
    
  pil_image = pt_to_pil(image)
  #pipe.watermarker.apply_watermark(pil_image, pipe.unet.config.sample_size)
  gen_image = pil_image[0]
  gen_image.save(dir_gen2 + files[i][:-4] + "genv2.jpg")
  v2_images.append(image)

In [ ]:
shutil.make_archive('/content/genvr2', 'zip', '/content/genvr2')
files.download("/content/genvr2.zip")

In [ ]:
del pipe
flush()

from diffusers.pipelines.deepfloyd_if import IFWatermarker

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler", 
    torch_dtype=torch.float16, 
    device_map="auto"
)

In [ ]:
dir_gen3 = "/content/genvr3/"
if not os.path.exists(dir_gen3):
  os.makedirs(dir_gen3)

for i in range(len(v2_images)):
  v2_image = v2_images[i]
  image_id = files_list[i][0:22]
  prompt = prompts[image_id]
  pil_image = pipe(prompt, generator=generator, image=v2_image).images
  #watermarker = IFWatermarker.from_pretrained("DeepFloyd/IF-I-XL-v1.0", subfolder="watermarker")
  #watermarker.apply_watermark(pil_image, pipe.unet.config.sample_size)
  gen_image = pil_image[0]
  gen_image.save(dir_gen3 + files_list[i][:-4] + "genv3.jpg")

In [ ]:
shutil.make_archive('/content/genvr3', 'zip', '/content/genvr3')
files.download("/content/genvr3.zip")